<a href="https://colab.research.google.com/github/PercyAyimbilaNsolemna/Machine_Learning/blob/main/DataCamp_KNN_Customer_Churn/DataCamp_Churn_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DATACAMP COURSE_1: CLASSIFICATION**
This notebook implements the k nearest classifier used in the classification course to develop a model for the churn dataset.

This implementation uses the K Neighbor Classifier from scikit learn to develop the model for the problem.

A simple Neural Network implementation will be available in a later notebook to defferiate how ML models compare with Neural Network models


In [161]:
#Imports numpy
import numpy as np
#Imports pandas
import pandas as pd
#Import the train_test_split from sklearn model selection
from sklearn.model_selection import train_test_split
#Imports the K Neighbour Classifier from sklearn neighbors
from sklearn.neighbors import KNeighborsClassifier
#Imports OneHotEncoder from sklearn preprocessing
from sklearn.preprocessing import OneHotEncoder

## **IMPORT GOOGLE DRIVE**

This imports google drive to the project so that the dataset for the project can be accessed and used for training, cross-validating and testing our model

In [162]:
#Imports drive from google colab
from google.colab import drive

In [163]:
#Connects google drive to the google colab notebook
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **TELCO CUSTOMER CHURN**

The Telco Custome Churn dataset will be used to develop the ML model.

The dataset can be downloaded [here](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)

Thanks to [kaggle](https://www.kaggle.com/)

In [164]:
#Loads the training data from the google drive using pandas
data = pd.read_csv('/content/drive/MyDrive/Datasets/Telco-Customer-Churn.csv')

## **DATA VISUALIZATION**

The dimensions or shape and some few training examples in our dataset will be explored.

In [165]:
#Outputs the type of data been used
print(f'The type of data used is \n {type(data)}')

The type of data used is 
 <class 'pandas.core.frame.DataFrame'>


In [166]:
#Output information about the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [167]:
#Outputs the shape or dimentions of the training data
print(f'The shape of the training data is {data.shape}')

The shape of the training data is (7043, 21)


In [168]:
#Outputs all the columns of the training data
#Includes both the features and the target
print(f'The training data has {len(data.columns)} columns') #data.columns for all the columns
print(f'\nThe columns are: ')
for column in data.columns:
  print(column)

The training data has 21 columns

The columns are: 
customerID
gender
SeniorCitizen
Partner
Dependents
tenure
PhoneService
MultipleLines
InternetService
OnlineSecurity
OnlineBackup
DeviceProtection
TechSupport
StreamingTV
StreamingMovies
Contract
PaperlessBilling
PaymentMethod
MonthlyCharges
TotalCharges
Churn


In [169]:
#Prints the first five rows of the training data
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [170]:
#Prints the last five rows of the data
data.tail()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes
7042,3186-AJIEK,Male,0,No,No,66,Yes,No,Fiber optic,Yes,...,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),105.65,6844.5,No


In [171]:
#Drops any row with a null value
data = data.dropna()
#data = data[~data['TotalCharges'] == ' ']
#Drops any row with an empty string ' ' as the value
data = data[~(data == ' ').any(axis=1)]
print(f'The shape of the training data after dropping all rows with an empty string is {data.shape}')

The shape of the training data after dropping all rows with an empty string is (7032, 21)


## **SPLITTING DATA INTO FEATURES AND TARGET VALUES**

This section splits the training data into training features and target values that is X and y

In [172]:
'''
#Features
X = data.drop(['customerID', 'Churn'], axis=1)
print(f'The shape of the training data is {X.shape} \n')

#Target value
y = data['Churn']
print(f'The shape of the target variable is {y.shape}')
y.head()
'''

"\n#Features\nX = data.drop(['customerID', 'Churn'], axis=1)\nprint(f'The shape of the training data is {X.shape} \n')\n\n#Target value\ny = data['Churn']\nprint(f'The shape of the target variable is {y.shape}')\ny.head()\n"

In [173]:
#Outputs the distinct or unique values in the churn column
print(f'The distinct columns in the {data.columns[20]} column are: \n {data[data.columns[20]].unique()} \n')
#Loops through the columns excluding the customer ID and displays the distinct values in each column
for column_index in range(1, len(data.columns)):
  print(f'The distinct values in the \033[1m {data.columns[column_index]} \033[0;0m column are: ')
  print(f'{data[data.columns[column_index]].unique()} \n')

The distinct columns in the Churn column are: 
 ['No' 'Yes'] 

The distinct values in the  gender  column are: 
['Female' 'Male'] 

The distinct values in the  SeniorCitizen  column are: 
[0 1] 

The distinct values in the  Partner  column are: 
['Yes' 'No'] 

The distinct values in the  Dependents  column are: 
['No' 'Yes'] 

The distinct values in the  tenure  column are: 
[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39] 

The distinct values in the  PhoneService  column are: 
['No' 'Yes'] 

The distinct values in the  MultipleLines  column are: 
['No phone service' 'No' 'Yes'] 

The distinct values in the  InternetService  column are: 
['DSL' 'Fiber optic' 'No'] 

The distinct values in the  OnlineSecurity  column are: 
['No' 'Yes' 'No internet service'] 

The distinct values in the  OnlineBackup  column are: 
['Ye

## **ONE-HOT ENCODING OF CATEGORICAL FEATURES**

In this section all the features with categorical data(text) will be converted to a vector of 0s and 1s.

*For example*

1. The **Churn feature** with two disinct values **YES** and **NO** will be encoded as **1** for **YES**  and **0** for **NO**

2. The **Internet Service feature** with three distinct values **DSL**, **Fiber
Optic** and **No** will be decoded as a **1*3 vector**. <br>
A customer who has uses **DSL** will be encoded as **[1 0 0]**, a customer who uses **Fibre Optic** will be encoded as **[0 1 0]** and a customer who does **NOT** use any of the Internet Service Providers will be encoded as **[0 0 1]**.

The coilumns to be transformed are listed below
* **Gender**
* **Partner**
* **Dependents**
* **Phone Service**
* **Multiple Lines**
* **Internet Service**
* **Online Security**
* **Online Backup**
* **Device Protection**
* **Tech Support**
* **Streaming TV**
* **Streaming Movies**
* **Contract**
* **Paperless Billing**
* **Payment method**
* **Total Charges**
* **Churn**

In [174]:
#Drops the customerID column from the data
data = data.drop(['customerID'], axis=1)

In [175]:

#Cast the Total charges column from object to float to a float
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])
#Displays the data types held by each of the columns in the data
data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [176]:
#Creates an object from the OneHotEncoding class
oneHotEncoder = OneHotEncoder()


In [177]:
#Use the fit_transform method in OneHotEncoder class to convert the categorical data into numerical data
transformed_features = oneHotEncoder.fit_transform(data[data.select_dtypes(include=['object']).columns]).toarray()
print(f'The shape of the transformed encoded features is {transformed_features.shape}')
#Converts the transformed features from float to int
transformed_features = transformed_features.astype(int)
#Outputs the one hot encoded features
print(transformed_features)

The shape of the transformed encoded features is (7032, 43)
[[1 0 0 ... 0 1 0]
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 0 1]
 ...
 [1 0 0 ... 0 1 0]
 [0 1 0 ... 1 0 1]
 [0 1 1 ... 0 1 0]]


In [178]:
#Extracts the one hot encoded features
encoded_features = oneHotEncoder.get_feature_names_out()
#Outputs the one hot encoded features
print(encoded_features)
print(f'\nThe total number of encoded features are {len(encoded_features)}')

['gender_Female' 'gender_Male' 'Partner_No' 'Partner_Yes' 'Dependents_No'
 'Dependents_Yes' 'PhoneService_No' 'PhoneService_Yes' 'MultipleLines_No'
 'MultipleLines_No phone service' 'MultipleLines_Yes'
 'InternetService_DSL' 'InternetService_Fiber optic' 'InternetService_No'
 'OnlineSecurity_No' 'OnlineSecurity_No internet service'
 'OnlineSecurity_Yes' 'OnlineBackup_No' 'OnlineBackup_No internet service'
 'OnlineBackup_Yes' 'DeviceProtection_No'
 'DeviceProtection_No internet service' 'DeviceProtection_Yes'
 'TechSupport_No' 'TechSupport_No internet service' 'TechSupport_Yes'
 'StreamingTV_No' 'StreamingTV_No internet service' 'StreamingTV_Yes'
 'StreamingMovies_No' 'StreamingMovies_No internet service'
 'StreamingMovies_Yes' 'Contract_Month-to-month' 'Contract_One year'
 'Contract_Two year' 'PaperlessBilling_No' 'PaperlessBilling_Yes'
 'PaymentMethod_Bank transfer (automatic)'
 'PaymentMethod_Credit card (automatic)' 'PaymentMethod_Electronic check'
 'PaymentMethod_Mailed check' 'Chu

In [179]:
#Creating a DataFrame for the encoded features
one_hot_encoded_features = pd.DataFrame(transformed_features, columns = encoded_features)
print(one_hot_encoded_features)

      gender_Female  gender_Male  Partner_No  Partner_Yes  Dependents_No  \
0                 1            0           0            1              1   
1                 0            1           1            0              1   
2                 0            1           1            0              1   
3                 0            1           1            0              1   
4                 1            0           1            0              1   
...             ...          ...         ...          ...            ...   
7027              0            1           0            1              0   
7028              1            0           0            1              0   
7029              1            0           0            1              0   
7030              0            1           0            1              1   
7031              0            1           1            0              1   

      Dependents_Yes  PhoneService_No  PhoneService_Yes  MultipleLines_No  \
0         

In [180]:
#Outputs the head of the one hot encodfed features
one_hot_encoded_features.head()

,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
0,1,0,0,1,1,0,1,0,0,1,...,0,0,0,1,0,0,1,0,1,0
1,0,1,1,0,1,0,0,1,1,0,...,1,0,1,0,0,0,0,1,1,0
2,0,1,1,0,1,0,0,1,1,0,...,0,0,0,1,0,0,0,1,0,1
3,0,1,1,0,1,0,1,0,0,1,...,1,0,1,0,1,0,0,0,1,0
4,1,0,1,0,1,0,0,1,1,0,...,0,0,0,1,0,0,1,0,0,1


In [181]:
#Outputs the tail of the one hot encoded features
one_hot_encoded_features.tail()

,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
7027,0,1,0,1,0,1,0,1,0,0,...,1,0,0,1,0,0,0,1,1,0
7028,1,0,0,1,0,1,0,1,0,0,...,1,0,0,1,0,1,0,0,1,0
7029,1,0,0,1,0,1,1,0,0,1,...,0,0,0,1,0,0,1,0,1,0
7030,0,1,0,1,1,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,1
7031,0,1,1,0,1,0,0,1,1,0,...,0,1,0,1,1,0,0,0,1,0


In [182]:
data.tail()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7038,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes
7042,Male,0,No,No,66,Yes,No,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),105.65,6844.50,No


In [183]:
# Reset indices to ensure alignment before concatinating to prevent NA in some of the cells
data.reset_index(drop=True, inplace=True)
one_hot_encoded_features.reset_index(drop=True, inplace=True)

print(data.shape)
print(one_hot_encoded_features.shape)

(7032, 20)
(7032, 43)


In [184]:
#Concatenates the one hot encoded features to the main training data
data_new = pd.concat([data, one_hot_encoded_features], axis=1)
data_new

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,0,0,0,1,0,0,1,0,1,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,1,0,1,0,0,0,0,1,1,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,0,0,0,1,0,0,0,1,0,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,1,0,1,0,1,0,0,0,1,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,...,1,0,0,1,0,0,0,1,1,0
7028,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,...,1,0,0,1,0,1,0,0,1,0
7029,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,...,0,0,0,1,0,0,1,0,1,0
7030,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,...,0,0,0,1,0,0,0,1,0,1


In [185]:
#Outputs the shape of the new DataFrame, which has the original features excluding the cunstomerID and the one hot encoded features
print(f'The shape of the new DataFrame is {data_new.shape}')

The shape of the new DataFrame is (7032, 63)


In [186]:
#Outputs the features of the new training data
print(data_new.columns)

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn', 'gender_Female',
       'gender_Male', 'Partner_No', 'Partner_Yes', 'Dependents_No',
       'Dependents_Yes', 'PhoneService_No', 'PhoneService_Yes',
       'MultipleLines_No', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'OnlineBackup_No',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'DeviceProtection_No', 'DeviceProtection_No internet service',
       'DeviceProtection_Yes', 'TechSupport_No',
       'TechSupport_No internet service

In [187]:
#Outputs the datatype of each of the features to be sure all are in discrete
data_new.dtypes

gender                                   object
SeniorCitizen                             int64
Partner                                  object
Dependents                               object
tenure                                    int64
                                          ...  
PaymentMethod_Credit card (automatic)     int64
PaymentMethod_Electronic check            int64
PaymentMethod_Mailed check                int64
Churn_No                                  int64
Churn_Yes                                 int64
Length: 63, dtype: object

In [188]:
#Outputs the head of the concatenated training data
data_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,0,0,0,1,0,0,1,0,1,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,1,0,1,0,0,0,0,1,1,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,0,0,0,1,0,0,0,1,0,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,1,0,1,0,1,0,0,0,1,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,0,0,0,1,0,0,1,0,0,1


In [189]:
#Outputs the tail of the new DataFrame
data_new.tail()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
7027,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,...,1,0,0,1,0,0,0,1,1,0
7028,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,...,1,0,0,1,0,1,0,0,1,0
7029,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,...,0,0,0,1,0,0,1,0,1,0
7030,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,...,0,0,0,1,0,0,0,1,0,1
7031,Male,0,No,No,66,Yes,No,Fiber optic,Yes,No,...,0,1,0,1,1,0,0,0,1,0


In [190]:
data.tail()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7027,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7028,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7029,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7030,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes
7031,Male,0,No,No,66,Yes,No,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),105.65,6844.50,No


## **FEATURE ENGINEERING**

This section tackles selecting relevant features in the concatenated dataFrame for training the Machine Learning Model.

**Gender:** The Gender feature will be retained with 1 referring to a customer being a male and a zero representing a customer beinng a female. The gender_male and gender_female columns will be removed. Conincisely, the **male column** will be converted to the **gender column**.

**Partner:** The Partner feature will be ratained with 1 being a customer with a partner **Yes** and 0 for a customer without a partner **No**

**Dependent:** A customer **with a dependent** will be saved as **1** and a customer **without a dependent** will be saved as **0**.

**Phone Service:** Customers **with Phone Service** will be saved as **1** whiles those **without a Phone Service** will be saved as **0**.

**Paperless Billing:** Customers with **Paperless Billing** will be saved as **1** whiles those **without Paperless Billing** will be saved as **0**.

**Churn:** Customers that churned will be saved as 1 and customers that did not churn will be saved as 0.

**Multiple Lines:** The **Multiple Lines** column will be **removed** leaving the **one hot encoded fields.**

The following listed columns will be removed leaving the one hot encoded features
**Internet Service**, **Online Security**, **Online BackUp**, **Device Protection**, **Tech Support**, **Streaming TV**, **Streaming Movies**, **Contract**, **Payment Method**.

In [191]:
print(data_new.columns)

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn', 'gender_Female',
       'gender_Male', 'Partner_No', 'Partner_Yes', 'Dependents_No',
       'Dependents_Yes', 'PhoneService_No', 'PhoneService_Yes',
       'MultipleLines_No', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'OnlineBackup_No',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'DeviceProtection_No', 'DeviceProtection_No internet service',
       'DeviceProtection_Yes', 'TechSupport_No',
       'TechSupport_No internet service

In [192]:
#Drop all the aforementioned fields
#Drops the columns inplace. That is no new DataFrame is created
data_new.drop(['gender', 'gender_Female', 'Partner', 'Partner_No', 'Dependents', 'Dependents_No', 'PhoneService', 'PhoneService_No', 'PaperlessBilling',
               'PaperlessBilling_No','Churn','Churn_No', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
               'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod'], axis = 1, inplace = True)

In [193]:
#Outputs the columns in the data_new DataFrame
print(data_new.columns)

Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges',
       'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes',
       'MultipleLines_No', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'OnlineBackup_No',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'DeviceProtection_No', 'DeviceProtection_No internet service',
       'DeviceProtection_Yes', 'TechSupport_No',
       'TechSupport_No internet service', 'TechSupport_Yes', 'StreamingTV_No',
       'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaperlessBilling_Yes',
       'PaymentMethod_Bank transfer (automatic)',
 

In [194]:
#Renames the columns that had binary labels to their original column names
#That is, Gender, Partner, Dependents, Phone Service, Paperless Billing, Churn
data_new.rename(columns = {'gender_Male': 'Gender', 'Partner_Yes': 'Partner', 'Dependents_Yes': 'Dependents', 'PhoneService_Yes': 'PhoneService',
                           'PaperlessBilling_Yes': 'PaperlessBilling', 'Churn_Yes': 'Churn', 'tenure': 'Tenure'}, inplace = True)

In [195]:
#Outputs the columns of the data_new DataFrame
print(data_new.columns)

Index(['SeniorCitizen', 'Tenure', 'MonthlyCharges', 'TotalCharges', 'Gender',
       'Partner', 'Dependents', 'PhoneService', 'MultipleLines_No',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'OnlineBackup_No', 'OnlineBackup_No internet service',
       'OnlineBackup_Yes', 'DeviceProtection_No',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No', 'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingTV_No', 'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaperlessBilling',
       'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit 

In [196]:
#Outputs the head of the data_new
data_new.head()

,SeniorCitizen,Tenure,MonthlyCharges,TotalCharges,Gender,Partner,Dependents,PhoneService,MultipleLines_No,MultipleLines_No phone service,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,0,1,29.85,29.85,0,1,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
1,0,34,56.95,1889.50,1,0,0,1,1,0,...,0,0,1,0,0,0,0,0,1,0
2,0,2,53.85,108.15,1,0,0,1,1,0,...,0,1,0,0,1,0,0,0,1,1
3,0,45,42.30,1840.75,1,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
4,0,2,70.70,151.65,0,0,0,1,1,0,...,0,1,0,0,1,0,0,1,0,1


In [197]:
#Outputs the tail of the data_new
data_new.tail()

,SeniorCitizen,Tenure,MonthlyCharges,TotalCharges,Gender,Partner,Dependents,PhoneService,MultipleLines_No,MultipleLines_No phone service,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
7027,0,24,84.80,1990.50,1,1,1,1,0,0,...,1,0,1,0,1,0,0,0,1,0
7028,0,72,103.20,7362.90,0,1,1,1,0,0,...,1,0,1,0,1,0,1,0,0,0
7029,0,11,29.60,346.45,0,1,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
7030,1,4,74.40,306.60,1,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,1
7031,0,66,105.65,6844.50,1,0,0,1,1,0,...,1,0,0,1,1,1,0,0,0,0


## **SPLITTING DATA INTO FEATURES AND TARGET VALUES**

This section splits the training data into training features and target values that is X and y

In [204]:
#Features and converts it to a numpy array
X = data_new.drop(['Churn'], axis=1).values
print(f'The shape of the training data is {X.shape} \n')
print(type(X))

#Target value and converts it to a numpy array
y = data_new['Churn'].values
print(f'The shape of the target variable is {y.shape}')

The shape of the training data is (7032, 40) 

<class 'numpy.ndarray'>
The shape of the target variable is (7032,)


## **SPLITTING THE TRAINING DATA**

This section splits the **data_new into training, cross-validation and testing** data using **train_test_split from scikit learn model selection**

In [ ]:
#Splits the data into training, cross-validation and testing
#TO BE CONTINUED!!!!

## **DEVELOPING THE MODEL**

This section deals with implementing the **K Nearest Neighbour(KNN)** using libraries from **scikit learn**.

Refer to the other notebook for the same model built using **Neural Network**